In [1]:
import serial
import random
import matplotlib.pyplot as plt
import time
import os
import numpy as np
import subprocess
import multiprocessing
from IPython.lib import backgroundjobs as bg
import h5py
import skimage
import sys
from skimage import io, transform, color, measure
from PIL import ImageTk, Image
import micro_functions as mf
import tkinter
import tkinter.messagebox
import tkinter.filedialog
import shared_variables as share
import hdf_functions as myh5
import gui_functions as gu
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
import cv2
from cv2 import MOTION_EUCLIDEAN
import scipy

In [2]:
cd "C:\Users\y7mandre\Pictures\digiCamControl\"

C:\Users\y7mandre\Pictures\digiCamControl


In [3]:
#mf.cyclecoms()
mf.opencoms()


Coms Open


In [4]:
share.zoomlevel=3
gu.gui_main()

Com Cycle Complete
startup commands complete


In [ ]:
plt.imshow(share.image_corrector)

In [ ]:
share.hf['imagedata/0/image_1'].attrs['position']

In [ ]:
image_1=share.hf['imagedata/0/image_0']
image_2=share.hf['imagedata/0/image_1']

In [ ]:
image_1.attrs['position']

In [ ]:
def bbox_extractor(image,position,pixelsize):
    #bbox is (left, right, top, bottom)
    
    image_bbox=np.zeros(4)
    image_bbox[0]=position[0]
    image_bbox[2]=position[1]
    image_bbox[1]=position[0]+image.shape[1]*pixelsize
    image_bbox[3]=position[1]+image.shape[0]*pixelsize
    image_bbox_pix=image_bbox/pixelsize
    
    return image_bbox, image_bbox_pix

In [ ]:
def find_overlap(image_1bbox,image_2bbox,pixelsize):
    bbox_lis=[image_2bbox,image_1bbox]
    delx1=np.absolute(image_1bbox[1]-image_2bbox[0])
    delx2=np.absolute(image_2bbox[1]-image_1bbox[0])
    delxs=[delx1,delx2]
    minxpos=np.argmin(delxs)
    minxval=np.amin(delxs)
    dely1=np.absolute(image_1bbox[3]-image_2bbox[2])
    dely2=np.absolute(image_2bbox[3]-image_1bbox[2])
    delys=[dely1,dely2]
    minypos=np.argmin(delys)
    minyval=np.amin(delys)
    overlap=np.zeros(4)
    overlap[0]=bbox_lis[minxpos][0]
    overlap[1]=bbox_lis[minxpos][0]+minxval
    overlap[2]=bbox_lis[minypos][2]
    overlap[3]=bbox_lis[minypos][2]+minyval
    overlap_pix=overlap/pixelsize
    return overlap, overlap_pix

In [ ]:
def extract_overlapimage(image,image_bbox,overlap,pixelsize):
    extractor_slicesx=([overlap[0]-image_bbox[0],overlap[1]-image_bbox[0]]/pixelsize).astype('int')
    extractor_slicesy=([overlap[2]-image_bbox[2],overlap[3]-image_bbox[2]]/pixelsize).astype('int')
    image_red=image[extractor_slicesy[0]:extractor_slicesy[1],extractor_slicesx[0]:extractor_slicesx[1],:]
    return image_red

In [ ]:
extractor_slicesx=([overlap[0]-image_1bbox[0],overlap[1]-image_1bbox[0]]/image_1.attrs['pixelsize']).astype('int')
extractor_slicesy=([overlap[2]-image_1bbox[2],overlap[3]-image_1bbox[2]]/image_1.attrs['pixelsize']).astype('int')
image_red=image_1[extractor_slicesy[0]:extractor_slicesy[1],extractor_slicesx[0]:extractor_slicesx[1],:]

In [ ]:
plt.imshow(image_red)

In [ ]:
print(extractor_slicesx,extractor_slicesy)

In [ ]:
extractor_slicesy.astype('int')

In [ ]:
image_1bbox,image_1bbox_pix=bbox_extractor(image_1,image_1.attrs['position'],image_1.attrs['pixelsize'])
image_2bbox,image_2bbox_pix=bbox_extractor(image_2,image_2.attrs['position'],image_2.attrs['pixelsize'])
overlap,overlap_pix=find_overlap(image_1bbox,image_2bbox,image_1.attrs['pixelsize'])
image_red1=extract_overlapimage(image_1,image_1bbox,overlap,image_1.attrs['pixelsize'])
image_red2=extract_overlapimage(image_2,image_2bbox,overlap,image_2.attrs['pixelsize'])

In [ ]:
t1=time.time()
image_red2_warped,warpmatrix_red1=mf.register_images_ECC(image_red1,image_red2,warp_matrix=wpmatrix,me=cv2.MOTION_TRANSLATION)
t2=time.time()
print(t2-t1)

In [ ]:
t1=time.time()
image_red2_warped,warpmatrix_red2=mf.register_images_ECC(image_red1,image_red2,warp_matrix=wpmatrix,me=cv2.MOTION_AFFINE)
t2=time.time()
print(t2-t1)

In [ ]:
print(warpmatrix_red1,warpmatrix_red2)

In [ ]:
wpmatrix=np.eye(2,3, dtype=np.float32)
print(wpmatrix)

In [ ]:
testim2=cv2.warpAffine(testim2, (wpmatrix), (testim1.shape[1],testim1.shape[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

In [ ]:
plt.imshow(im2warped)

In [ ]:
t1=time.time()
im2warped1,warpmatrix1=mf.register_images_ECC_euclid(image_red1,image_red2)
t2=time.time()
print(t2-t1)

In [ ]:
im1reg,h=mf.register_images_ORB(image_red1,image_red2)

In [ ]:
wpmatrix=np.eye(2, 3, dtype=np.float32)
wpmatrix[0,2]=-overlap_pix[0]
wpmatrix[1,2]=-overlap_pix[2]
print(wpmatrix)

In [ ]:
overlap_pix

In [ ]:
diff=np.subtract(im2warped,image_red1,dtype='float')

In [ ]:
diff=np.absolute(diff).astype('uint8')

In [ ]:
plt.imshow(diff*20)

In [ ]:
fig,ax=plt.subplots(figsize=(16,24))
ax.set_xlim(0,image_2bbox_pix[1])
plt.imshow(image_1,extent=image_1bbox_pix,origin='lower')
plt.imshow(image_2,extent=image_2bbox_pix,alpha=1,origin='lower')
plt.imshow(image_red,extent=overlap_pix,origin='lower')
rect=patches.Rectangle((overlap_pix[0],overlap_pix[2]),(overlap_pix[1]-overlap_pix[0]),(overlap_pix[3]-overlap_pix[2]),color='red',alpha=0.3)
ax.add_patch(rect)
plt.show()

In [ ]:
plt.imshow(image_red)

In [ ]:
print(overlap_pix[0],overlap_pix[2],overlap_pix[1]-overlap_pix[0])

In [ ]:
image_1.attrs['pixelsize']

In [ ]:
plt.imshow(imlist[1])
for x in range(len(kp1)):
    plt.scatter(kp1[x].pt[0],kp1[x].pt[1],c='red')
for x in range(len(kp2)):
    plt.scatter(kp2[x].pt[0],kp2[x].pt[1],c='blue')